## Description
This is an initial notebook to apply the ADF tests to time-series data.

$\Delta y(t) = \lambda y(t-1) + \mu + \beta t + \alpha_1\Delta y(t-1) + ... + \alpha_k\Delta y(t-k) + \epsilon_k$

In [1]:
import pandas as pd
import plotly.express as px
import yfinance as yf

from statsmodels.tsa.stattools import adfuller

In [2]:
asset_name = 'ISCTR.IS'
analysis_period = '17y'
normalization = True
normalization_asset_name = 'USDTRY=X'


In [3]:
main_asset = yf.download(asset_name, period = analysis_period)
plot_y = [f'{asset_name}']
if normalization:
    normalizaton_asset = yf.download(normalization_asset_name, period = analysis_period)
    
    assets = pd.merge_asof(normalizaton_asset['Close'], main_asset['Adj Close'], on='Date')
    assets = assets.rename(columns = {'Close' : f'{normalization_asset_name[:-2]}', 'Adj Close' : f'{asset_name}'})
    assets[f'{asset_name}_norm'] = assets[f'{asset_name}']/assets[f'{normalization_asset_name[:-2]}']
    
    plot_y.extend([f'{normalization_asset_name[:-2]}',f'{asset_name}_norm'])
    display(f'{len(main_asset)} and {len(normalizaton_asset)}')
assets.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


'4363 and 4427'

,Date,USDTRY,ISCTR.IS,ISCTR.IS_norm
0,2008-02-14,1.1897,0.426259,0.358291
1,2008-02-15,1.1925,0.415235,0.348206
2,2008-02-18,1.1845,0.429934,0.362967
3,2008-02-19,1.1957,0.444633,0.371860
4,2008-02-20,1.2077,0.433608,0.359036


In [4]:
fig = px.line(assets, 
              x = 'Date', 
              y = plot_y, 
              log_y=True
              )
fig.update_layout(xaxis_title = 'Date',
                  legend_title = 'Asset',
                  yaxis_title = 'Value'
                  )
display(fig)

# ADF Test

In [5]:
if normalization:
    adfuller(assets[f'{asset_name}_norm'])
else:
    adfuller(assets[f'{asset_name}'])

The p-value (~0.1605...) of the ADF test is bigger than 0.05. Therefore, the null hypothesis cannot be rejected, which suggests that the investigated time-series data is non-stationary.

In [6]:
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [7]:
series = pd.DataFrame()
series['asset_value'] = np.array(assets[f'{asset_name}_norm'][:4426])
series['asset_diff'] = np.array(assets[f'{asset_name}_norm'].diff(periods=1).iloc[1:])
series.head()

,asset_value,asset_diff
0,0.358291,-0.010085
1,0.348206,0.014761
2,0.362967,0.008893
3,0.371860,-0.012823
4,0.359036,0.012232


In [8]:
model = sm.OLS(series['asset_diff'],
               series['asset_value']
               )
results = model.fit()
optimization_parameters = results.params
half_life = -np.log(2)/optimization_parameters.asset_value

display(f'Half life to trade the stock, {asset_name}, is {half_life:.2f} days.')

'Half life to trade the stock, ISCTR.IS, is 2659.37 days.'